In [1]:
# import the libraries
from pathlib import Path
import plotly.express as px
import dash
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
import pycountry_convert as pc

In [2]:
# Load the Dataframe
df = pd.read_csv(Path("./ds_salaries.csv"))

In [3]:
# Preview the dataset
df.head()

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


In [4]:
# convert employee_residence column from ISO 3166-1 alpha-2 to ISO 3166-1 alpha-3
# Define pycountry-convert function
def convert(row):
    #convert alpha-2 to country name
    country_names = pc.country_alpha2_to_country_name(row.employee_residence, "default")
    
    #convert from country name to alpha-3
    alpha3 = pc.country_name_to_country_alpha3(country_names, "default")
    
    return alpha3

# overwrite existing values with alpha-3 country codes
df["employee_residence"] = df.apply(convert, axis = 1)


In [5]:
df.head()

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DEU,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JPN,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GBR,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HND,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,USA,50,US,L


In [6]:
# Create the dash app
app = JupyterDash(__name__)

In [7]:
# set up graphs
fig = px.histogram(df, x="employee_residence")
fig2 = px.choropleth(df, locations="employee_residence",
                     color="employee_residence", hover_name="employee_residence")


# Set up the app layout - dropdown for job title
app.layout = html.Div(children=[
    html.H1('Data Science Job Analyzer', style={'textAlign':'center'}),
    html.Br(),
    dcc.Dropdown(
        id='job-dropdown',
        options=df["job_title"].unique(),
        style={"width": "50%", "offset":1,},
        clearable=False
    ),
    dcc.Graph(id='hist', figure=fig),
    # New Div for all elements in the new 'row' of the page
html.Div([
        dcc.Graph(id='map', figure=fig2),  
    ]),
])

In [8]:
# Callback function
@app.callback(
    Output(component_id="hist", component_property="figure"),
    Output(component_id="map", component_property="figure"),
    Input(component_id="job-dropdown", component_property="value")
)

def update_hist(selected_job_title):
    filtered_df = df[df.job_title == selected_job_title]
    fig = px.histogram(filtered_df, x="employee_residence")
    fig2 = px.choropleth(filtered_df, locations="employee_residence",
                     color="employee_residence", hover_name="employee_residence")
    return fig, fig2


In [9]:
# Run local server
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
